Loading data from csv

In [ ]:
import pandas as pd
data = pd.read_csv('../data/cleanedHouses.csv', sep=',')

training model and finding the best mse and combination

In [ ]:
from sklearn.model_selection import train_test_split
import itertools
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler

input_features = ["lat", "lon", "usable_area", "land_area","garage","new","furnished","cellar","parkingLots","reconstructed"]
target_feature = "price"
bestRmse = None
bestCombination = None
bestModel = None
scaler = StandardScaler()

for r in range(1, len(input_features) + 1):
    for combo in itertools.combinations(input_features, r):
        X_train, X_test, y_train, y_test = train_test_split(data[list(combo)], data[target_feature], test_size=0.1)
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.fit_transform(X_test)
        base_model = DecisionTreeRegressor(max_depth=3)
        model = AdaBoostRegressor(estimator=base_model, n_estimators=50)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        if(bestRmse is None or (mse**(1/2))<bestRmse):
            bestRmse = mse**(1/2)
            bestCombination = list(combo)
            bestModel = model

print(bestRmse)
print(bestCombination)

Saving model to dat for later use

In [ ]:
import pickle
import json

with open("../SavedModels/AdaBoostRegressor.dat","wb") as f:
    pickle.dump(bestModel,f)

with open("../SavedModels/AdaBoostRegressor.json","w") as f:
    json.dump(bestCombination,f)